In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import kagglehub
import os

In [3]:
data_folder = './Data/'
movie_data = pd.read_csv(data_folder+'clean_data_original.csv')
budget_data = pd.read_csv(data_folder+'movie_data_tmbd.csv', sep='|', engine='python')


movie_data.shape

(81513, 12)

In [5]:
#remove the lines where the budget is equal to 0 and nan because it means for both that we don't have an information on the budget
budget_data['budget'] = pd.to_numeric(budget_data['budget'], errors='coerce')

budget_data_clean = budget_data[budget_data['budget'].notna() & (budget_data['budget'] != 0)]
budget_data_clean = budget_data_clean.drop_duplicates(subset='imdb_id')
budget_data_clean.shape

(14074, 27)

In [ ]:
# now merging the two data sets

movie_data.rename(columns={'Runtime':'runtime'}, inplace=True)

merged_data_budget = movie_data.merge(budget_data_clean[['title', 'runtime', 'budget']], 
                               on=['title', 'runtime'], 
                               how='inner')

merged_data_budget.shape
merged_data_budget.head(5)

(3519, 13)

In [9]:
imdb_data = pd.read_csv(data_folder+'clean_data_imdb.csv')

imdb_data.rename(columns={'BoxOfficeRevenue':'BoxOfficeRevenueImdb'}, inplace=True)

#merge the imdb data with the merged data and only add the colum BoxofficerevenueImdb and the column ratingStandardize to the merged data
imdb_data.rename(columns={'Runtime':'runtime'}, inplace=True)



merged_data_imdb = merged_data_budget.merge(imdb_data[['title', 'BoxOfficeRevenueImdb', 'RatingStandardize', 'runtime']],
                       on=['title','runtime'], 
                       how='inner')

merged_data_imdb.shape

(3031, 15)

In [10]:
merged_data_imdb_clean = merged_data_imdb[merged_data_imdb['BoxOfficeRevenueImdb'].notna() | merged_data_imdb['BoxOfficeRevenue'].notna()]

# now for all the lines where there is a value for BoxOfficeRevenueImdb and not for BoxOfficeRevenue we will copy the value from BoxOfficeRevenueImdb to BoxOfficeRevenue and vice versa

merged_data_imdb_clean['BoxOfficeRevenue'] = merged_data_imdb_clean['BoxOfficeRevenue'].fillna(merged_data_imdb_clean['BoxOfficeRevenueImdb'])
merged_data_imdb_clean['BoxOfficeRevenueImdb'] = merged_data_imdb_clean['BoxOfficeRevenueImdb'].fillna(merged_data_imdb_clean['BoxOfficeRevenue'])

# now for all the lines where there is a value for BoxOfficeRevenueImdb and for BoxOfficeRevenue we will calculate the difference in percentage between the two values and put it in a new column called BoxOfficeRevenueDifference
def percentage_difference(X, Y):                                           

    return np.abs((X-Y))/np.maximum(X,Y) * 100

merged_data_imdb_clean['BoxOfficeRevenueDifference'] = percentage_difference(merged_data_imdb_clean['BoxOfficeRevenue'], merged_data_imdb_clean['BoxOfficeRevenueImdb'])

# print the mean of the BoxOfficeRevenueDifference column

print(merged_data_imdb_clean['BoxOfficeRevenueDifference'].mean())


merged_data_imdb_clean.sample(5)

0.08997550131860477


C:\Users\mehdi\AppData\Local\Temp\ipykernel_31656\2059975187.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_imdb_clean['BoxOfficeRevenue'] = merged_data_imdb_clean['BoxOfficeRevenue'].fillna(merged_data_imdb_clean['BoxOfficeRevenueImdb'])
C:\Users\mehdi\AppData\Local\Temp\ipykernel_31656\2059975187.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_imdb_clean['BoxOfficeRevenueImdb'] = merged_data_imdb_clean['BoxOfficeRevenueImdb'].fillna(merged_data_imdb_clean['BoxOfficeRevenue

,Unnamed: 0,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,runtime,Languages,Countries,Genres,Year,BoxOfficeStandardize,budget,BoxOfficeRevenueImdb,RatingStandardize,BoxOfficeRevenueDifference
2394,64280,2390149,/m/078m6t,Corky Romano,2001,23900000.0,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/0gw5w78"": ""G...",2001.0,0.008587,11000000.0,23900000.0,0.47,0.0
1261,33668,66853,/m/0hmm7,The Conversation,1974-04-07,4420000.0,113.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02n4kr"": ""Myster...",1974.0,0.001585,1600000.0,4420000.0,0.77,0.0
1494,40713,58018,/m/0fxb2,Tora! Tora! Tora!,1970,29548291.0,144.0,"{""/m/03_9r"": ""Japanese Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/0gw5n2f"": ""Japanese Movies"", ""/m/02p0szs""...",1970.0,0.010617,25485000.0,29548291.0,0.75,0.0
2755,74174,4836948,/m/0cq8nx,Mrs. Miniver,1942-06-04,8878000.0,134.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/05swd"": ""Propaganda film"", ""/m/02h8pkk"": ...",1942.0,0.003187,1344000.0,8878000.0,0.76,0.0
1090,29072,452767,/m/02vzxsf,Underworld,2003-09-19,95708457.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",2003.0,0.034396,22000000.0,95708457.0,0.70,0.0


In [11]:
merged_data_imdb_clean = merged_data_imdb_clean.drop_duplicates(subset='Wiki_ID')

merged_data_imdb_clean.shape


(1904, 16)